In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to("cuda")


# Thai → Vietnamese
source_lang = "tha"
target_lang = "vie"
forced_bos_id = 662  # Vietnamese bos token ID



In [9]:
from tqdm import tqdm
import torch
import time
from torch.cuda.amp import autocast


with open("/home/leloc/Document/USTH/Thesis/Data/flores200_dataset/devtest/tha_Thai.devtest", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

# Dịch theo batch
batch_size = 8
outputs = []

start_time = time.time()

# Dịch và ghi kết quả
for i in tqdm(range(0, len(lines), batch_size), desc="Dịch câu"):
    batch = lines[i:i+batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to("cuda")
    
    with autocast():
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=forced_bos_id,
            max_length=256
        )
    translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    outputs.extend(translations)

    # Log thời gian ước tính
    elapsed_time = time.time() - start_time
    print(f"Đã xử lý {i+batch_size}/{len(lines)} câu. Thời gian đã trôi qua: {elapsed_time:.2f} giây")

# Sau khi hoàn thành
print("Dịch xong!")


# Lưu kết quả
output_path = "/home/leloc/Document/USTH/Thesis/Results/flores200_tha2vie_output.txt"
with open(output_path, "w", encoding="utf-8") as f:
    for line in outputs:
        f.write(line.strip() + "\n")

print(f"✅ Kết quả đã lưu tại: {output_path}")

Dịch câu:   0%|          | 0/127 [00:00<?, ?it/s]

/tmp/ipykernel_6929/3590541142.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Dịch câu:   1%|          | 1/127 [00:41<1:26:27, 41.17s/it]

Đã xử lý 8/1012 câu. Thời gian đã trôi qua: 41.17 giây


Dịch câu:   2%|▏         | 2/127 [01:42<1:50:56, 53.26s/it]

Đã xử lý 16/1012 câu. Thời gian đã trôi qua: 102.89 giây


...

Đã xử lý 24/1012 câu. Thời gian đã trôi qua: 168.26 giây


Dịch câu:   3%|▎         | 4/127 [03:05<1:27:12, 42.54s/it]

Đã xử lý 32/1012 câu. Thời gian đã trôi qua: 185.89 giây


Dịch câu:   3%|▎         | 4/127 [03:09<1:36:57, 47.30s/it]


KeyboardInterrupt: 